In [261]:
import numpy as np
import os
import cv2
from pathlib import Path
from os.path import exists
from os import remove

In [262]:
#folder where the image is located
folder_name = "no"

#parent folder
data_path = Path("D:\\Projektarbete\\blandat\\brain_tumor_dataset")

#filname including path
path_no = list((data_path / folder_name).rglob('*'))

In [263]:
type(path_no)

list

In [264]:
def check_duplicates(image_list):
    """In lista med bilder och ut en dict med duplicerade bilder"""
    
    list_of_pixel = []          #lista totalt antal pixlar per bild
    unique_pixel = set()        #lista unika pixel värden
    list_of_duplicate_pixel = []        #lista duplicerade pixel värden
    dict_duplicates = {}                #dictionary av de duplicerade värdena(value) och path (key)

    #för varje bild i image_list lägg till summerade pixelvärdet i list_of_pixel    
    for i in range(len(image_list)):
        list_of_pixel.append(np.sum(image_list[i]))
    
    #för varje pixelvärde i list_of_pixel lägg unika värden i unique_pixel och duplicerade i duplicate_pixel_pos
    for num in list_of_pixel:
        if num not in unique_pixel:
            unique_pixel.add(num)
        else:
            unique_pixel = unique_pixel - set([num])
            list_of_duplicate_pixel.append(num)
    
    #för varje pixelvärde i image_list som finns i duplicated_pixel_pos spara path (key) och pixelvärde (value) i en dict
    for i in range(len(image_list)):
        if np.sum(image_list[i]) in list_of_duplicate_pixel:
            dict_duplicates[path_no[i]] = os.stat(path_no[i]).st_size
    
    return dict_duplicates

In [265]:
image_list = []
IMG_SIZE = (224,224)

#spara varje bild som en numpy array i en lista
for path in path_no:
    img = cv2.imread(str(path))
    img = cv2.resize(img, IMG_SIZE)
    image_list.append(img)

In [266]:
#funktionen för att få ut duplicerade bilder
duplicates = check_duplicates(image_list)

In [267]:
def sortera_dict(sorted_dict):
    """Får in ett dict och skickar ut ett sorterat dict"""
    sorted_dict = {}
    sorted_keys = sorted(duplicates, key=duplicates.get)  

    for w in sorted_keys:
        sorted_dict[w] = duplicates[w]
    
    return sorted_dict

In [268]:
#sorterar vårt dict (dupplicerade bilder) efter pixelvärde
sorted_dict = sortera_dict(duplicates)
print(len(sorted_dict))

22


In [269]:
def which_files_delete(sorted_dict):
    """In vår dict med dupplicerade värden, ut 2 listor, en med unika värden och en med de filer som skall tas bort. """
    listan = []
    listan_pixel = []
    listan_delete = []

    for key in sorted_dict:
        if sorted_dict[key] not in listan_pixel:
            listan_pixel.append(sorted_dict[key])
            listan.append(key)
        else:
            listan_delete.append(key) 
        
    print(f"Filer att spara: {len(listan)}")
    print(f"Filer att ta bort: {len(listan_delete)}")

    return listan, listan_delete
    

In [270]:
#kör funktionen för att få en lista på de filer som skall tas bort
listan, listan_delete = which_files_delete(sorted_dict)

Filer att spara: 11
Filer att ta bort: 11


In [271]:
#lägger ihop "parent folder" + duplicate_folder + namnet på mappen som körs
new_folder = data_path.joinpath("duplicate_folder\\"+folder_name+"\\")

#skapar mapp om det inte finns
if not exists(new_folder):
    os.makedirs(new_folder)

In [272]:
def move_and_delete_dup(new_folder, listan_delete):
    """funktionen skapar en kopia av bilder som skall tas bort och flyttar dessa till en mapp
    sedan tas dubbletter bort från originalmappen"""
    filename_number = 0
    nr_files_moved = 0

    for file in listan_delete:
        #path och filnamn och filtyp
        filename = str(new_folder)+"\\"+str(filename_number)+'img.jpg'
        filename_number +=1
        
        #läs bild
        img = cv2.imread(str(file))
        
        #spara dubblett
        saving_image = cv2.imwrite(filename, img)
        
        #ta bort dubblett från orginalmapp
        if exists(filename):
            remove(str(file))
            nr_files_moved += 1
        else:
            print("Couldnt find",filename)
    print(nr_files_moved)

In [273]:
#Skapa kopior och spara i ny mapp och ta bort dubbletter 
move_and_delete_dup(new_folder, listan_delete)

11
